# Natural Language Database Queries with Zero Shot RoBERTa-based SQL Query Generation

Based on the Google Colab Notebook (https://colab.research.google.com/drive/1qYJTbbEXYFVdY6xae9Zmt96hkeW8ZFrn) provided for the paper "[Data Agnostic RoBERTa-based Natural Language to SQL Query Generation](https://arxiv.org/abs/2010.05243)". Only a list of fields and their types are needed for the model to convert Natural Language to an SQL query.

This notebook shows how to implement the model to make a Natural Language to SQL converter for any uploaded tabular dataset.

Key Funtionality

* Allows csv file upload
* Loads the csv into an in-memory SQLite database
* Determines the type schema automatially
* Loads the pre-trained models
* Converts Natural Language to SQL
* Executes the SQL against the in-memory SQLite database
* Display the results in a filterable table

Note:
You will need to download the model files from [here](https://drive.google.com/drive/folders/13f2MrdpieC9QGXM_DJnj2f1Hs6ZBh2ZT?usp=sharing.) and upload to your own Google Drive. You will need to update the path to the location within your Google Drive. The 2 required files are model_roberta_best.pt and model_best.pt.




In [2]:
# Install libs
!pip install tableschema
!pip install sqlalchemy
!pip install records
!pip install transformers==3.4.0

     |████████████████████████████████| 71kB 4.9MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 133kB 12.1MB/s 
     |████████████████████████████████| 133kB 13.7MB/s 
     |████████████████████████████████| 245kB 15.7MB/s 
     |████████████████████████████████| 7.4MB 14.6MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-cp37-none-any.whl size=10768 sha256=497d1335ede663e9df8f7e2974b72e4953166899aa5312dc1c284e770148d1e6
  Stored in directory: /root/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-cp37-none-any.whl size=7383 sha256=c4a9be91e214053e7fe3986c6b2eb78d8f3e1402dea036fb537d57978d2ddba4
  Stored in directory: /root/.cache/pip/wheels/3f/8a/cb/38917fd1ef4356b9870ace7331b83417dc594bf2c029bd991f
Successfully built unicodec

In [3]:
# Imports
import csv
from tableschema import infer
import io
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from google.colab import files
from google.colab import data_table

# Load Google Collab Extensions
%load_ext google.colab.data_table

In [4]:
# upload csv file
uploaded = files.upload()

Saving Datasets for DesignLAK 2021 - ds1.csv to Datasets for DesignLAK 2021 - ds1.csv


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Set path to pre-trained models stored on your Google Drive
path_wikisql = "/content/drive/My Drive/2021Explorations/NL2SQL/RoBERTa NL2SQL"


In [7]:
def getSchema(filename):
  schema_types = []
  field_names = []
  schema = infer(filename, limit=500, headers=1, confidence=0.85)
  field_objs = schema['fields']
  for field in field_objs:
    field_names.append(field['name'])
    schema_type = field['type']
    if schema_type == 'string':
      schema_types.append('text')
    else:
      schema_types.append('real')
  return {'schema_types': schema_types,'field_names':field_names}

sqlite_db = create_engine('sqlite://',echo=False)

uploaded_files = list(uploaded.keys())

uploaded_file = None
schema_types = []
field_names = []

if len(uploaded_files) > 0 :
  uploaded_file = uploaded_files[0]
  schema = getSchema(uploaded_file)
  schema_types = schema['schema_types']
  field_names = schema['field_names']

  # Add data to in memory sqllite database
  with open(uploaded_file, 'r') as file:
    data_df = pd.read_csv(file)
    data_df.to_sql('uploadedtable', con=sqlite_db, index=True, index_label='uploaded_id', if_exists='replace')
else:
  print('No file has been uploaded')


In [8]:
# Adapted from  https://colab.research.google.com/drive/1qYJTbbEXYFVdY6xae9Zmt96hkeW8ZFrn but with the training and testing removed
# Just the loading of pre-trained model and converting NL to SQL
# The orignal github repo is https://github.com/DebadityaPal/RoBERTa-NL2SQL
# https://github.com/aneesha/RoBERTa-NL2SQL is a fork with code changes to return SQL (rather than printing it out) and add quotes around string/text fields

!rm -rf RoBERTa-NL2SQL

GIT_PATH = "https://github.com/aneesha/RoBERTa-NL2SQL"
!git clone "{GIT_PATH}"
%cd RoBERTa-NL2SQL

import load_data
import torch
import json,argparse
import load_model
import roberta_training
import corenlp_local
import seq2sql_model_testing
import seq2sql_model_training_functions
import model_save_and_infer
import dev_function
import infer_functions
import time
import os
import nltk

from dbengine_sqlnet import DBEngine
from torchsummary import summary
from tqdm.notebook import tqdm
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda")

# load models
roberta_model, tokenizer, configuration = load_model.get_roberta_model()          # Loads the RoBERTa Model
seq2sql_model = load_model.get_seq2sql_model(configuration.hidden_size) 

path_roberta_pretrained = path_wikisql + "/model_roberta_best.pt"
path_model_pretrained = path_wikisql + "/model_best.pt"

# load pre-trained weights
if torch.cuda.is_available():
    res = torch.load(path_roberta_pretrained)
else:
    res = torch.load(path_roberta_pretrained, map_location='cpu')

roberta_model.load_state_dict(res['model_roberta'])

if torch.cuda.is_available():
    res = torch.load(path_model_pretrained)
else:
    res = torch.load(path_model_pretrained, map_location='cpu')

seq2sql_model.load_state_dict(res['model'])


Cloning into 'RoBERTa-NL2SQL'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 38 (delta 13), reused 6 (delta 3), pack-reused 0
Unpacking objects: 100% (38/38), done.
/content/RoBERTa-NL2SQL
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<All keys matched successfully>

In [18]:
table_id = 'uploadedtable'

# Sample queries
#nlu = "How many students?"
#nlu = "What is the highest value in 'Quiz 1'?"

#nlu = "How many students received a 'Quiz 1' score greater than 90?"

#nlu = "How many students received a 'Quiz 1' score greater than 50 and less than 70?"
#nlu = "Find students that received a 'Quiz 1' score greater than 50 and less than 70?"

#nlu = "What is the average value in 'Quiz 1'?"

#@title ## Enter a Natural Language Query
#@markdown SQL will be generated and the returned data will be generated

#Find students have a gender equal to Female?

#@markdown ---
natural_language_query = "Show unique languages" #@param {type:"string"}
#@markdown ---

domainswap =['score']

if 'score' in natural_language_query:
  natural_language_query = natural_language_query.replace('score', 'value')

sqlqueries = infer_functions.infer(
                natural_language_query,
                table_id, field_names, schema_types, tokenizer, 
                seq2sql_model, roberta_model, configuration, max_seq_length=222,
                num_target_layers=2,
                beam_size=4
            )

sqlquery = sqlqueries[0]
print('Generated SQL: ',sqlquery)

aggs = ['count', 'avg', 'max', 'min', 'distinct']
uniquelist = ['distinct','unique']


if any([x in natural_language_query for x in uniquelist]):
  sqlquery = sqlquery.replace('SELECT ', 'SELECT distinct ')

if not any(x in sqlquery for x in aggs):
  sqlquery = sqlquery.replace('SELECT ', 'SELECT *, ')

print('Postprocessed SQL: ',sqlquery)

df = pd.read_sql(sqlquery, sqlite_db)
data_table.DataTable(df, include_index=False, num_rows_per_page=20)

Generated SQL:  SELECT (Language) FROM uploadedtable
Postprocessed SQL:  SELECT distinct (Language) FROM uploadedtable


,Language
0,Mandarin
1,English
